In [4]:
import pandas as pd
import geopandas as gpd

How many Native Americans live in the United States?

In [5]:
nation = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/census-data-downloader/master/data/processed/acs5_2017_aianaloneorincombo_nationwide.csv",
    dtype={"geoid": str}
)

In [6]:
nation

,geoid,name,universe,us
0,1,United States,5487131.0,1


How many Native Americans live on a reservation?

In [7]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/datadesk/census-data-downloader/master/data/processed/acs5_2017_aianaloneorincombo_aiannhhomeland.csv",
    dtype={"geoid": str}
)

In [8]:
df.head()

,geoid,name,universe,american indian area/alaska native area/hawaiian home land
0,0125,"Augustine Reservation, CA",0.0,125
1,0155,"Barona Reservation, CA",557.0,155
2,0185,Benton Paiute Reservation and Off-Reservation ...,110.0,185
3,0120,Auburn Rancheria and Off-Reservation Trust Lan...,0.0,120
4,0115,"Aroostook Band of Micmac Trust Land, ME",184.0,115


In [9]:
def get_geotype(geoid):
    val = int(geoid)
    if val < 5000:
        return 'Federal reservation or trust land'
    elif val < 5500:
        return 'Hawaiian home land'
    elif val < 6000:
        return 'Oklahoma tribal statistical area'
    elif val < 8000:
        return 'Alaska native village statistical area'
    elif val < 9000:
        return 'Tribal-designated statistical area'
    elif val < 9500:
        return 'State reservation'
    elif val < 9999:
        return 'State-designated tribal statistical area'

In [10]:
df['geotype'] = df.geoid.apply(get_geotype)

In [11]:
df.geotype.value_counts(dropna=False)

Federal reservation or trust land           326
Alaska native village statistical area      220
Hawaiian home land                           75
State-designated tribal statistical area     30
Oklahoma tribal statistical area             29
State reservation                            10
Tribal-designated statistical area            4
Name: geotype, dtype: int64

In [12]:
trimmed = df[[
    'geoid',
    'geotype',
    'name',
    'universe',
]]

In [13]:
trimmed.head()

,geoid,geotype,name,universe
0,0125,Federal reservation or trust land,"Augustine Reservation, CA",0.0
1,0155,Federal reservation or trust land,"Barona Reservation, CA",557.0
2,0185,Federal reservation or trust land,Benton Paiute Reservation and Off-Reservation ...,110.0
3,0120,Federal reservation or trust land,Auburn Rancheria and Off-Reservation Trust Lan...,0.0
4,0115,Federal reservation or trust land,"Aroostook Band of Micmac Trust Land, ME",184.0


In [15]:
typetotals = trimmed.groupby("geotype").agg({
    "geoid": "count",
    "universe": "sum"
})

In [16]:
typetotals

,geoid,universe
geotype,,
Alaska native village statistical area,220,82749.0
Federal reservation or trust land,326,587200.0
Hawaiian home land,75,588.0
Oklahoma tribal statistical area,29,420377.0
State reservation,10,736.0
State-designated tribal statistical area,30,102291.0
Tribal-designated statistical area,4,898.0


If 13.8% of the reservation population was missed in 2010, what does that pencil out to?

In [17]:
571000 * 0.138

78798.0

What are the biggest reservations?

In [18]:
reservations = trimmed[trimmed.geotype == 'Federal reservation or trust land']

In [22]:
reservations.sort_values("universe", ascending=False).head(10)

,geoid,geotype,name,universe
175,2430,Federal reservation or trust land,Navajo Nation Reservation and Off-Reservation ...,168725.0
204,2810,Federal reservation or trust land,"Pine Ridge Reservation, SD--NE",17447.0
81,1140,Federal reservation or trust land,"Fort Apache Reservation, AZ",14669.0
90,1310,Federal reservation or trust land,"Gila River Indian Reservation, AZ",10664.0
220,3355,Federal reservation or trust land,"San Carlos Reservation, AZ",10275.0
230,3235,Federal reservation or trust land,Rosebud Indian Reservation and Off-Reservation...,10220.0
172,2595,Federal reservation or trust land,"Osage Reservation, OK",10169.0
63,1110,Federal reservation or trust land,"Flathead Reservation, MT",9596.0
287,4200,Federal reservation or trust land,Tohono O'odham Nation Reservation and Off-Rese...,9464.0
311,4785,Federal reservation or trust land,Zuni Reservation and Off-Reservation Trust Lan...,9169.0


In [105]:
gdf = gpd.read_file("./data/tiger/tl_2018_us_aiannh/tl_2018_us_aiannh.shp")

In [106]:
deduped_gdf = gdf.dissolve(by='AIANNHCE').reset_index()

In [109]:
trimmed_gdf = deduped_gdf[[
    'AIANNHCE',
    'geometry'
]].rename(columns={
    "AIANNHCE": "geoid"
})

In [110]:
merged_gdf = trimmed_gdf.merge(
    reservations,
    on="geoid",
    how="inner"
)

In [111]:
assert len(merged_gdf) == len(reservations)

In [112]:
merged_gdf.to_crs(epsg=4326).to_file("./data/analysis/reservations-polygons.geojson", driver="GeoJSON")

In [113]:
merged_gdf['geometry'] = merged_gdf.geometry.centroid

In [114]:
merged_gdf.to_crs(epsg=4326).to_file("./data/analysis/reservations-points.geojson", driver="GeoJSON")